In [1]:
import pandas as pd
import json
import sqlite3
import numpy as np
from datetime import datetime

In [2]:
conn = sqlite3.connect('../data/recsys.db')
c = conn.cursor()

## Interaction Data

We want to separate the user data into those that exist before 2013 (training) and those that exist after 2013 (test). We remove users with less than 3 ratings in each of those time bins. In addition, we want to use this data to simulate paired choice behavior (that is given 2 recipes, which item will a person choose) and so we need some amount of variation in the data. That is each user needs more than 1 unique rating value. So we also exclude any user where all the ratings are the same (0 variance).

In [46]:
c.execute('''DROP TABLE IF EXISTS pre_interactions;''')
c.execute('''DROP TABLE IF EXISTS post_interactions;''')
c.execute('''/* Pre-2013 - User Ids - Variance of Ratings > 0 */
CREATE TEMPORARY TABLE pre_interactions AS
SELECT user_id, (AVG(rating*rating) - AVG(rating)*AVG(rating)) AS var_rating, COUNT(*) AS n_ratings
FROM interactions
WHERE (dateLastModified - strftime('%s','2013-01-01')) < 0
GROUP BY user_id
HAVING var_rating > 0 AND n_ratings > 2;''')
c.execute('''/* Post-2013 - User Ids - Variance of Ratings > 0 */
CREATE TEMPORARY TABLE post_interactions AS
SELECT user_id, (AVG(rating*rating) - AVG(rating)*AVG(rating)) AS var_rating, COUNT(*) AS n_ratings
FROM interactions
WHERE (dateLastModified - strftime('%s','2013-01-01')) >= 0
GROUP BY user_id
HAVING var_rating > 0 AND n_ratings > 2;''')
conn.commit()

In [47]:
df = pd.read_sql_query('''/* Ok so now I want those user_ids that exist both pre-2013 and post-2013 */
/* And then I want to get all the ratings for those user-ids */
SELECT user_id, recipe_id, rating, dateLastModified AS date_int, datetime(dateLastModified, 'unixepoch') AS date, 
	(CASE WHEN (dateLastModified - strftime('%s','2013-01-01')) < 0 THEN 1 ELSE 0 END) AS training
FROM interactions
	INNER JOIN pre_interactions AS prei
		USING (user_id)
	INNER JOIN post_interactions AS posti
		USING (user_id)
ORDER BY user_id, date ASC;''', conn)
df.head()

,user_id,recipe_id,rating,date_int,date,training
0,32,9076,5,950969193,2000-02-19 14:06:33,1
1,32,15901,5,951595599,2000-02-26 20:06:39,1
2,32,9584,5,951595951,2000-02-26 20:12:31,1
3,32,13949,5,979319716,2001-01-12 17:15:16,1
4,32,14494,5,979319823,2001-01-12 17:17:03,1


In [48]:
df.groupby('user_id').agg('count')

,recipe_id,rating,date_int,date,training
user_id,,,,,
32,85,85,85,85,85
39,293,293,293,293,293
338,289,289,289,289,289
935,23,23,23,23,23
1005,16,16,16,16,16
...,...,...,...,...,...
17875901,12,12,12,12,12
18131839,28,28,28,28,28
18220416,9,9,9,9,9


In [49]:
df.groupby('training').agg(['count', 'nunique'])

user_id         recipe_id          rating         date_int          \
           count nunique     count nunique   count nunique    count nunique   
training                                                                      
0         187660   13026    187660   34620  187660       5   187660  187476   
1         404762   13026    404762   30572  404762       5   404762  404270   

            date          
           count nunique  
training                  
0         187660  187476  
1         404762  404270

In [50]:
404762/(404762 + 187660) # percent of training

0.6832325605733751